# DS-поток, весна 2025
## Задание ADL.4
### Подходы к оптимизации процесса обучения LLMs.

**Правила:**

* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Выполненную работу нужно отправить телеграм-боту `@miptstats_ds24_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. Дождитесь подтверждения от бота, что он принял файл. Если подтверждения нет, то что-то не так. **Работы, присланные иным способом, не принимаются.**
* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb`**.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания получат штраф.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* В каждой задаче не забывайте делать **пояснения и выводы**.
* **Код из рассказанных на занятиях ноутбуков** можно использовать без ограничений.

**Баллы за задание:**

* Реализация &mdash; 80 баллов;
* Сравнение и анализ &mdash; 70 баллов.

<b><font size="5">Дорешка:</font></b>

* <b><font size="4">Задача 1</font></b>
  * <b><font color="#00b565" size="3">Исправил код</font></b>

*Примечание*
* <b><font color="#00b565">#00b565 &mdash; цвет для исправления замечаний</font></b>
* <b><font color="#f09000">#f09000 &mdash; цвет для решений с нуля</font></b>
* проверяющий может иначе классифицировать эти два типа дорешек

In [115]:
# Bot check

# HW_ID: ds_adl4
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [116]:
import time
import torch
import torch.nn as nn
from collections import defaultdict

## Описание

На занятии мы познакомились с различными техниками, которые используются при обучении больших языковых моделей. В этом домашнем задании вам предстоит решить реальную практическую задачу, которая связана с оптимизацией некоторых слоев в трансформере.

Вспомним общую идею техники, которая называется Gradient Checkpointing. Идея заключается в том, чтобы на этапе forward'a не запоминать промежуточные активации, необходимые для backward'a, а вычислять их непосредственно на этапе backward'a. Почему это может быть важно? Оказывается, что активации MLP для больших моделей занимают очень много памяти. Сохранить для backward'а все активации, включая слои внимания, просто невозможно. Возникает вопрос: стоит ли сохранять промежуточные активации MLP или же отдать память под активации attention'a? На практике пересчет активаций для MLP оказывается гораздо быстрее, чем пересчет того же attention'a. В итоге мы можем не сохранять активации MLP, экономить достаточно много памяти, а часть освободившейся памяти отдать под активации attention'a и тем самым даже ускорить обучение!

## Реализация
Сегодня мы не будем работать с полноценным трансформером, а сфокусируемся только на MLP-блоке. Вам предлагается написать модифицированный слой MLP таким образом, чтобы он поддерживал возможность либо сохранять промежуточные активации, либо пересчитывать их на этапе backward'a. Оформить код нужно будет в виде кастомной `torch.autograd.Function`. Хорошая практика заключается в том, чтобы ваш итоговый слой, который наследуется от `torch.nn.Module` просто вызывал функцию с нужными параметрами. Вам нужно **обязательно** ознакомиться с [постом](https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html), в котором вы узнаете, как правильно написать кастомную `torch.autograd.Function` функцию и что это вообще такое.


Вспомним с лекции, как выглядит модифицированный MLP для современных арихетктур.
$$
\text{FFN}_{\text{SwiGLU}}\left(x, W, V, U\right) = \left(\text{Swish}_1\left(xW\right)\otimes xV\right)U = \left(\text{SiLU}\left(xW\right)\otimes xV\right)U,
$$
где
$$
\text{Swish}_1\left(x\right)= \text{SiLU}\left(x\right) = x\sigma\left(x\right)
$$

Такой MLP-блок используется в [LLaMA](https://arxiv.org/abs/2307.09288)-подобных архитектурах. В этом задании будем использовать его. Посмотрим, как выглядит реализация в виде простого `torch.nn.Module`.

<b><font color="#00b565" size="5">Исправил код</font></b>

<b><font color="#00b565" size="3">Замечание:</font></b> <i>  
Реализация.  
▫️ Наличие решения. (80.0)  
▫️ Неправильно реализован подсчет градиента для матрицы U. Инд. комм. строка silu_xW = torch.sigmoid(xW)*(xV) должна быть c xW, а не xV (-20.0)  
▫️ Неправильно реализован подсчет градиента для матрицы W. Инд. комм. тут тоже прокиывается ошибка из-за silu_xw  
▫️ Неправильно реализован подсчет градиента для матрицы V. Инд. комм. тут тоже прокиывается ошибка из-за silu_xw  
Баллы: 60.0 / 80.0</i>  

In [117]:
class SwigluMLP(nn.Module):
    def __init__(self, hidden_size, intermediate_size):
        super().__init__()
        self.W = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.V = nn.Linear(hidden_size, intermediate_size, bias=False)
        self.U = nn.Linear(intermediate_size, hidden_size, bias=False)
        self.act_fn = nn.SiLU()
    def forward(self, x):
        output = self.U(self.act_fn(self.W(x)) * self.V(x))
        return output

Теперь перейдем к нашей реализации. Обратите внимание, что `MemoryOptimizedSwigluMLPFunction` принимает на вход `checkpoint_level`. Это переменная нужна для реализации следующей логики:
* `checkpoint_level == 0` &mdash; никаких оптимизаций не проводится, промежуточные активации просто сохраняются для переиспользования на этапе backward'a через `ctx.save_for_backward(...)`;
* `checkpoint_level == 1` &mdash; для backward'a сохраняются только вход `x` и матрицы `W, V, U`, а на этапе backward'a нужные активации просто снова пересчитываются.

Вам нужно реализовать методы `forward` и `backward`. Реализация второго потребует от вас посчитать некоторые промежуточные градиенты в матричном виде. Обязательно **выпишите и поясните** получающиеся формулы.

$\frac{d}{dx}SiLU(x) = x' \sigma(x) + x \sigma'(x) = \sigma(x) + x\sigma(x)(1 - \sigma(x))$  
$\frac{d}{dW}out = \frac{d}{dW} ((SiLU(xW) \otimes xV)U) = ( \frac{d SiLU(xW)}{d W} \otimes xV ) U = ( x ( \sigma(xW) + xW \sigma(xW) (1 - \sigma(xW)) ) \otimes xV ) U$  
$\frac{d}{dV}out = \frac{d}{dV} ((SiLU(xW) \otimes xV)U) = (SiLU(xW) \otimes \frac{d (xV)}{dV}) U = (SiLU(xW) \otimes x) U$  
$\frac{d}{dU}out = \frac{d}{dU} ((SiLU(xW) \otimes xV)U) = (SiLU(xW) \otimes xV)$  
*Пояснение: chain rule. Надеюсь, расписал без опечаток)*

In [ ]:
class MemoryOptimizedSwigluMLPFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, W, V, U, checkpoint_level):  # x размера (batch_size,seq_len,hidden_dim)
        xW,xV = torch.matmul(x,W), torch.matmul(x,V)
        silu_xW = torch.sigmoid(xW)*xW
        if checkpoint_level == 0:
            ctx.save_for_backward(x,W,V,U,xW,xV,silu_xW)
        else:
            ctx.save_for_backward(x,W,V,U)
        ctx.checkpoint_level = checkpoint_level
        return torch.matmul((silu_xW*xV),U)
    @staticmethod
    def backward(ctx, grad_output):
        x,W,V,U,*etc = ctx.saved_tensors
        if ctx.checkpoint_level == 0:
            xW,xV,silu_xW = etc
        else:
            xW,xV = torch.matmul(x,W), torch.matmul(x,V)
            silu_xW = torch.sigmoid(xW)*(xW)  # фикс
        grad_xW,grad_xV = torch.matmul(grad_output,U.t())*(torch.sigmoid(xW)+xW*torch.sigmoid(xW)*(1-torch.sigmoid(xW))), torch.matmul(grad_output,U.t())*silu_xW
        grad_W,grad_V,grad_U = x.transpose(1,2)@grad_xW, x.transpose(1,2)@grad_xV, (silu_xW*xV).transpose(1,2)@grad_output
        grad_x = torch.matmul(grad_xW,W.t())+torch.matmul(grad_xV,V.t())
        return grad_x,grad_W,grad_V,grad_U,None

class MemoryOptimizedSwigluMLP(nn.Module):  # оптимизированный MLP-слой
    def __init__(self, hidden_size, intermediate_size, checkpoint_level=0):
        super(MemoryOptimizedSwigluMLP, self).__init__()
        self.W = nn.Parameter(torch.randn(intermediate_size, hidden_size))
        self.V = nn.Parameter(torch.randn(intermediate_size, hidden_size))
        self.U = nn.Parameter(torch.randn(hidden_size, intermediate_size))
        self.checkpoint_level = checkpoint_level
    def forward(self, x):
        return MemoryOptimizedSwigluMLPFunction.apply(x,self.W,self.V,self.U,self.checkpoint_level)

Теперь проверим, что реализованный MLP-слой считается верно.

In [ ]:
batch_size, seq_len, hidden_dim = 4, 256, 768  # Зададим параметры

dummy_input = torch.randn(batch_size, seq_len, hidden_dim)
# Обратите внимание, что intermediate_size кратно больше hidden_dim
# Это типичное "расширение", характерное для MLP (FFN) слоя в трансформере
swiglu_mlp = SwigluMLP(hidden_dim, hidden_dim * 3)
optimized_swiglu_mlp = MemoryOptimizedSwigluMLP(hidden_dim, hidden_dim * 3)

with torch.no_grad():  # Скопируем параметры, чтобы они были одинаковые
    optimized_swiglu_mlp.W.data = swiglu_mlp.W.weight.data.t()
    optimized_swiglu_mlp.V.data = swiglu_mlp.V.weight.data.t()
    optimized_swiglu_mlp.U.data = swiglu_mlp.U.weight.data.t()

standard_output = swiglu_mlp(dummy_input)  # Прогоним модель
optimized_output = optimized_swiglu_mlp(dummy_input)
assert torch.allclose(standard_output, optimized_output, atol=1e-4)  # Проверим выходы слоев на совпадение

standard_output.sum().backward()
optimized_output.sum().backward()
assert torch.allclose(swiglu_mlp.U.weight.grad, optimized_swiglu_mlp.U.grad.t(), atol=1e-4)  # Проверим на совпадение градиенты
assert torch.allclose(swiglu_mlp.V.weight.grad, optimized_swiglu_mlp.V.grad.t(), atol=1e-4)
assert torch.allclose(swiglu_mlp.W.weight.grad, optimized_swiglu_mlp.W.grad.t(), atol=1e-4)

*Все работает*

<b><font color="#00b565" size="5">===== Конец исправлений =====</font></b>

## Сравнение и анализ

Сравните время исполнение forward/backward и объем потребляемой памяти в зависимости от значения `checkpoint_level`. Проведите эксперименты для разных значений `batch_size`, `seq_len`, `hidden_dim`. Сделайте запуски в нескольких сетапах. Попробуйте достаточно большие `seq_len=1024` и `hidden_dim=4096`, а также `num_layers=5`. Сделайте выводы.

Некоторые советы:
* Для отслеживания потребляемой памяти можете воспользоваться `torch.cuda.max_memory_allocated()`. Желательно после каждого шага очищать статистику, используя `torch.cuda.reset_peak_memory_stats()`. Более подробно рекомендуется почитать документацию по [ссылке](https://pytorch.org/docs/stable/notes/cuda.html#cuda-memory-management).
* Для подсчета времени можно воспользоваться простым ` time.perf_counter()` или же `time.time()`. Однако с подсчетом времени для GPU-операций все немного хитрее. В PyTorch и других библиотеках, работающих с GPU, операции выполняются асинхронно по отношению к коду, исполняемому на CPU. Такой подход позволяет CPU продолжать работу, не ожидая окончания каждой операции на GPU, что способствует повышению общей производительности за счёт параллельной работы CPU и GPU. Что это значит на практике? Вы можете получить завышенные результаты своих измерений, так как замер времени может завершиться до того, как GPU в действительности закончит выполнение операций.
Рассмотрим пример кода:
```
a = torch.rand(10000, 10000, device="cuda")
start_time = time.time()
b = a @ a
elapsed_time = time.time() - start_time
```
В этом примере, после запуска операции умножения матриц `a @ a`, мы немедленно измеряем время выполнения функции, не дожидаясь её фактического завершения на GPU.
Для получения точных измерений времени выполнения операций на GPU необходимо использовать синхронизацию. В PyTorch это можно сделать с помощью функции `torch.cuda.synchronize()`, которая блокирует выполнение кода на CPU до тех пор, пока все запланированные задачи на соответствующем GPU не будут завершены. \
Пример более грамотного кода:
```
a = torch.rand(10000, 10000, device='cuda')
torch.cuda.synchronize() # ждем завершения всех предыдущих операций на GPU
start_time = time.time()
b = a @ a
torch.cuda.synchronize() # cнова синхронизируемся, чтобы убедиться, что операция завершена
elapsed_time = time.time() - start_time
```
* Для подсчета статистики следует сделать несколько проходов forward/backward для одной модели, а полученные результаты просто усреднить. Для более стабильных результатов выполните также разогрев, то есть некоторое количество прогонов модели перед основным измерением. Это важно, т.к. на результаты измерений могут повлиять дополнительные задержки, связанные с инициализацией и загрузкой ресурсов, температурой GPU, а также различные кэши.
* Представьте результаты и выводы в информативном виде, хорошо подойдет какая-нибудь табличка. Затраты по памяти лучше всего указать в Гб, а время исполнения в секундах.

In [121]:
# используем несколько слоев, чтобы увидеть выигрыш по памяти
# в случае chechkpoint_level == 0 для вычислений очередного слоя будет использована та память, что осталась для предыдущего
# если же checkpoint_level == 1, то придется хранить активации для всех слоев

class TestTransformer(nn.Module):
    def __init__(self, num_layers, hidden_size, intermediate_size, checkpoint_level):
        super(TestTransformer, self).__init__()
        self.layers = nn.ModuleList([MemoryOptimizedSwigluMLP(hidden_size, intermediate_size, checkpoint_level) for _ in range(num_layers)])
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

Реализация функции для бенчмарка.

In [122]:
def benchmark_transformer(model, batch_size, seq_len, hidden_dim, num_warmup_steps, num_steps):
    statistics = defaultdict(list)
    input_tensor = torch.randn(batch_size, seq_len, hidden_dim, device='cuda')  # Генерация входных данных
    for _ in range(num_warmup_steps):  # Warmup GPU чтобы избежать заниженного перфоманса
        model(input_tensor)
        torch.cuda.synchronize()
    for _ in range(num_steps):  # Основное измерение
        torch.cuda.reset_peak_memory_stats()
        start = time.time()
        output = model(input_tensor)
        torch.cuda.synchronize()
        statistics['time_fwd'].append(time.time()-start)
        start = time.time()
        output.sum().backward()
        torch.cuda.synchronize()
        statistics['time_bwd'].append(time.time()-start)
        statistics['memory'].append(torch.cuda.max_memory_allocated() / (1024**3))  # в гб
    for k,v in statistics.items():
        statistics[k] = sum(v) / num_steps
    return statistics

In [123]:
batch_size, seq_len, hidden_dim, num_layers = 4, 1024, 4096, 5
stats = {'batch_size':[],'seq_len':[],'hidden_dim':[],'num_layers':[],'checkpoint_level':[],'time_fwd':[],'memory':[],'time_bwd':[]}
for bs in [2,4,8]:
    for seq_len in [256,512,1024]:
        for hid_dim in [1024,2048,4096]:
            for num_ls in [3,4,5]:
                for lvl in [0,1]:
                    model = TestTransformer(num_ls, hid_dim, hid_dim*3, lvl)
                    model.cuda()
                    for k,v in benchmark_transformer(model, bs, seq_len, hid_dim, 10, 50).items():
                        stats[k].append(v)
                    for k,v in {'batch_size':bs,'seq_len':seq_len,'hidden_dim':hid_dim,'num_layers':num_ls,'checkpoint_level':lvl}.items():
                        stats[k].append(v)
                    print(bs, seq_len, hid_dim, num_ls, lvl)

2 256 1024 3 0
2 256 1024 3 1
2 256 1024 4 0
2 256 1024 4 1
2 256 1024 5 0
2 256 1024 5 1
2 256 2048 3 0
2 256 2048 3 1
2 256 2048 4 0
2 256 2048 4 1
2 256 2048 5 0
2 256 2048 5 1
2 256 4096 3 0
2 256 4096 3 1
2 256 4096 4 0
2 256 4096 4 1
2 256 4096 5 0
2 256 4096 5 1
2 512 1024 3 0
2 512 1024 3 1
2 512 1024 4 0
2 512 1024 4 1
2 512 1024 5 0
2 512 1024 5 1
2 512 2048 3 0
2 512 2048 3 1
2 512 2048 4 0
2 512 2048 4 1
2 512 2048 5 0
2 512 2048 5 1
2 512 4096 3 0
2 512 4096 3 1
2 512 4096 4 0
2 512 4096 4 1
2 512 4096 5 0
2 512 4096 5 1
2 1024 1024 3 0
2 1024 1024 3 1
2 1024 1024 4 0
2 1024 1024 4 1
2 1024 1024 5 0
2 1024 1024 5 1
2 1024 2048 3 0
2 1024 2048 3 1
2 1024 2048 4 0
2 1024 2048 4 1
2 1024 2048 5 0
2 1024 2048 5 1
2 1024 4096 3 0
2 1024 4096 3 1
2 1024 4096 4 0
2 1024 4096 4 1
2 1024 4096 5 0
2 1024 4096 5 1
4 256 1024 3 0
4 256 1024 3 1
4 256 1024 4 0
4 256 1024 4 1
4 256 1024 5 0
4 256 1024 5 1
4 256 2048 3 0
4 256 2048 3 1
4 256 2048 4 0
4 256 2048 4 1
4 256 2048 5 0
4 256 2

OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 240.12 MiB is free. Process 6615 has 14.50 GiB memory in use. Of the allocated memory 13.62 GiB is allocated by PyTorch, and 771.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

*Памяти перестало хватать, посмотрим на то, что посчиталось*

In [128]:
import pandas as pd
for i in range(11):
    display(pd.DataFrame(stats).round(3).iloc[10*i:10*i+10])

,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
0,2,256,1024,3,0,0.009,3.104,0.024
1,2,256,1024,3,1,0.009,3.070,0.030
2,2,256,1024,4,0,0.011,3.194,0.029
3,2,256,1024,4,1,0.011,3.142,0.037
4,2,256,1024,5,0,0.013,3.283,0.035
5,2,256,1024,5,1,0.014,3.214,0.045
6,2,256,2048,3,0,0.037,4.025,0.094
7,2,256,2048,3,1,0.037,3.956,0.117
8,2,256,2048,4,0,0.041,4.345,0.112
9,2,256,2048,4,1,0.042,4.241,0.141


,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
10,2,256,2048,5,0,0.053,4.664,0.132
11,2,256,2048,5,1,0.053,4.526,0.166
12,2,256,4096,3,0,0.138,7.551,0.382
13,2,256,4096,3,1,0.137,7.413,0.466
14,2,256,4096,4,0,0.174,8.753,0.420
15,2,256,4096,4,1,0.175,8.546,0.535
16,2,256,4096,5,0,0.214,9.954,0.520
17,2,256,4096,5,1,0.213,9.679,0.656
18,2,512,1024,3,0,0.018,3.184,0.043
19,2,512,1024,3,1,0.018,3.115,0.055


,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
20,2,512,1024,4,0,0.021,3.292,0.054
21,2,512,1024,4,1,0.021,3.189,0.069
22,2,512,1024,5,0,0.026,3.401,0.065
23,2,512,1024,5,1,0.026,3.263,0.083
24,2,512,2048,3,0,0.071,4.184,0.175
25,2,512,2048,3,1,0.072,4.046,0.220
26,2,512,2048,4,0,0.086,4.542,0.206
27,2,512,2048,4,1,0.086,4.335,0.262
28,2,512,2048,5,0,0.107,4.899,0.244
29,2,512,2048,5,1,0.106,4.624,0.313


,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
30,2,512,4096,3,0,0.303,7.868,0.732
31,2,512,4096,3,1,0.300,7.592,0.932
32,2,512,4096,4,0,0.387,9.146,0.806
33,2,512,4096,4,1,0.386,8.732,1.072
34,2,512,4096,5,0,0.468,10.424,0.997
35,2,512,4096,5,1,0.465,9.873,1.319
36,2,1024,1024,3,0,0.035,3.344,0.084
37,2,1024,1024,3,1,0.036,3.204,0.107
38,2,1024,1024,4,0,0.043,3.492,0.106
39,2,1024,1024,4,1,0.043,3.282,0.136


,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
40,2,1024,1024,5,0,0.055,3.639,0.125
41,2,1024,1024,5,1,0.055,3.360,0.159
42,2,1024,2048,3,0,0.154,4.502,0.346
43,2,1024,2048,3,1,0.153,4.225,0.451
44,2,1024,2048,4,0,0.200,4.936,0.399
45,2,1024,2048,4,1,0.197,4.522,0.536
46,2,1024,2048,5,0,0.245,5.371,0.478
47,2,1024,2048,5,1,0.244,4.818,0.646
48,2,1024,4096,3,0,0.521,8.502,1.380
49,2,1024,4096,3,1,0.521,7.951,1.713


...

,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
60,4,256,2048,3,0,0.070,4.465,0.176
61,4,256,2048,3,1,0.071,4.327,0.221
62,4,256,2048,4,0,0.086,4.823,0.208
63,4,256,2048,4,1,0.086,4.616,0.262
64,4,256,2048,5,0,0.106,5.181,0.245
65,4,256,2048,5,1,0.107,4.905,0.315
66,4,256,4096,3,0,0.301,8.993,0.746
67,4,256,4096,3,1,0.301,8.717,0.950
68,4,256,4096,4,0,0.388,10.271,0.822
69,4,256,4096,4,1,0.385,9.857,1.087


,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
70,4,256,4096,5,0,0.468,11.549,1.012
71,4,256,4096,5,1,0.466,10.998,1.338
72,4,512,1024,3,0,0.036,3.414,0.084
73,4,512,1024,3,1,0.036,3.275,0.107
74,4,512,1024,4,0,0.043,3.562,0.107
75,4,512,1024,4,1,0.044,3.353,0.137
76,4,512,1024,5,0,0.056,3.710,0.124
77,4,512,1024,5,1,0.055,3.431,0.159
78,4,512,2048,3,0,0.154,4.783,0.351
79,4,512,2048,3,1,0.154,4.507,0.455


,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
80,4,512,2048,4,0,0.198,5.218,0.401
81,4,512,2048,4,1,0.199,4.803,0.546
82,4,512,2048,5,0,0.244,5.652,0.480
83,4,512,2048,5,1,0.244,5.100,0.654
84,4,512,4096,3,0,0.520,9.627,1.401
85,4,512,4096,3,1,0.520,9.076,1.734
86,4,512,4096,4,0,0.669,11.058,1.556
87,4,512,4096,4,1,0.670,10.231,1.998
88,4,512,4096,5,0,0.817,12.489,1.930
89,4,512,4096,5,1,0.817,11.387,2.459


,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
90,4,1024,1024,3,0,0.076,3.734,0.166
91,4,1024,1024,3,1,0.077,3.454,0.222
92,4,1024,1024,4,0,0.098,3.960,0.203
93,4,1024,1024,4,1,0.098,3.540,0.276
94,4,1024,1024,5,0,0.124,4.186,0.239
95,4,1024,1024,5,1,0.123,3.625,0.322
96,4,1024,2048,3,0,0.273,5.423,0.674
97,4,1024,2048,3,1,0.272,4.865,0.841
98,4,1024,2048,4,0,0.357,6.014,0.771
99,4,1024,2048,4,1,0.357,5.177,1.002


,batch_size,seq_len,hidden_dim,num_layers,checkpoint_level,time_fwd,memory,time_bwd
100,4,1024,2048,5,0,0.435,6.605,0.941
101,4,1024,2048,5,1,0.436,5.489,1.220
102,4,1024,4096,3,0,1.028,10.898,2.731
103,4,1024,4096,3,1,1.029,9.793,3.391
104,4,1024,4096,4,0,1.342,12.637,3.073
105,4,1024,4096,4,1,1.341,10.979,3.944


**Вывод:**

*В этой задаче удалось посмотреть, как разные параметры влияют на скорость и память в зависимости от характера чекпоинтинга (checkpoint_level). Метод очень удобный для огромных моделей: пересчитываем много тензоров, но зато экономим память*